In [2]:
import pandas as pd
import os
import re
import evaluate

toxicity = evaluate.load("toxicity", module_type="measurement")

#path = 'cleaned_transcript.txt'
path = 'acrimony.txt'

with open(path, 'r') as file:
    text = file.read()

timestamp_pattern = re.compile(r'\d{2}:\d{2}:\d{2},\d+ --> \d{2}:\d{2}:\d{2},\d+\n')
cleaned_text = re.sub(r'^\d+\n', '', text, flags=re.MULTILINE) # remove line numbers if any
cleaned_text = re.sub(timestamp_pattern, '', cleaned_text)
cleaned_text = '\n'.join(line for line in cleaned_text.splitlines() if line.strip())

# Split the text into rows
rows = cleaned_text.split('\n')

# Split each row into columns
data = [row.split('\t') for row in rows]


# Convert the data to a pandas dataframe
df = pd.DataFrame(data, columns=["Text"])

/Users/isab7070/anaconda3/envs/mynlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using default facebook/roberta-hate-speech-dynabench-r4-target checkpoint


In [3]:
# Loop through Text and run toxicity.compute()
results = []
for text in df['Text']:
    result = toxicity.compute(predictions=[text])
    results.append(result)

# Create a new dataframe with Text and Toxicity columns
df_results = pd.DataFrame({'Text': df['Text'], 'Toxicity': results})
df_results.to_csv('results_acrimony_movie.csv', index=False)

In [1]:
#df_results['Toxicity']

In [23]:
# Extract 'toxicity' values and store in a new column 'Toxicity_Value'
df_results['Toxicity_Value'] = df_results['Toxicity'].apply(lambda x: x['toxicity'][0])

# Print the resulting DataFrame
#print(df_results[['Toxicity', 'Toxicity_Value']])

mean_toxicity = df_results['Toxicity_Value'].mean()

print("Mean toxicity: ", mean_toxicity)

Mean toxicity:  0.025139345635293287
